## Imports

In [1]:
import numpy as np
import pandas as pd
import os
from preprocessing import Preprocessor
from dataset_balancing import Balance

In [44]:
import tensorflow as tf
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout, Bidirectional, LSTM, Input
from tensorflow.keras.metrics import Precision

In [55]:
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, log_loss, roc_curve, fbeta_score

## Data preprocessing

In [3]:
# Load training and test sets
df_train = pd.read_json('datas/training_set.json')
df_test = pd.read_json('datas/testing_set.json')

print(f"Train shape : {df_train.shape}")
print(f"Test shape : {df_test.shape}")

Train shape : (6035, 2)
Test shape : (1065, 2)


In [4]:
# Balance the train set
train_balance = Balance(df_train)
train_balance.process_balance('./preprocessed_data/train_balanced')

In [5]:
# Preprocess the data
train_preprocessor = Preprocessor(train_balance.df_dataset)
train_preprocessor.preprocess('./preprocessed_data/train_nlp_preprocessed')
test_preprocessor = Preprocessor(df_test)
test_preprocessor.preprocess('./preprocessed_data/test_nlp_preprocessed')

C:\Users\Lenovo\Anaconda3\envs\ingenierie_logicielle_ia\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


(15995, 93, 300)
(15995, 8)


C:\Users\Lenovo\Documents\Paul\Travail\ENSEIRB-MATMECA\3A\Ingenierie_logicielle_IA\projet_industrialisation_ia\preprocessing.py:123: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  data = np.array(data)


(1065, 92, 300)
(1065, 8)


In [13]:
train_preprocessor.df_dataset.head()

,sentence,intent
0,"[[-0.0033358, -8.6066, -7.4396, -3.3738, 12.51...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]"
1,"[[-3.398, 3.652, -11.154, -0.76505, 0.59124, -...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]"
2,"[[-0.78445, -3.7161, 1.5587, 3.4742, -2.844, 2...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]"
3,"[[-0.0033358, -8.6066, -7.4396, -3.3738, 12.51...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]"
4,"[[-1.0319, 4.7613, -7.8448, -1.0902, 1.3204, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]"


In [40]:
x_train = np.array(list(train_preprocessor.df_dataset['sentence']))
y_train = np.array(list(train_preprocessor.df_dataset['intent']))
x_test = np.array(list(test_preprocessor.df_dataset['sentence']))
y_test = np.array(list(test_preprocessor.df_dataset['intent']))

print("x_train.shape = ", x_train.shape)
print("y_train.shape = ", y_train.shape)
print("x_test.shape = ", x_test.shape)
print("y_test.shape = ", y_test.shape)

x_train.shape =  (15995, 93, 300)
y_train.shape =  (15995, 8)
x_test.shape =  (1065, 92, 300)
y_test.shape =  (1065, 8)


## Model creation and training

In [47]:
model = Sequential()
model.add(Input(shape=x_train[0].shape))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax'))

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_3 (Bidirection (None, 128)               186880    
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 8)                 520       
Total params: 199,816
Trainable params: 199,816
Non-trainable params: 0
________________________________________________

In [48]:
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=[Precision()]
)

history = model.fit(x_train, y_train, batch_size=128, epochs=20, verbose=1)

Epoch 1/20
125/125 [==============================] - 28s 138ms/step - loss: 1.9283 - precision: 0.6418
Epoch 2/20
125/125 [==============================] - 16s 130ms/step - loss: 0.9283 - precision: 0.8485
Epoch 3/20
125/125 [==============================] - 16s 130ms/step - loss: 0.4659 - precision: 0.9085
Epoch 4/20
125/125 [==============================] - 16s 132ms/step - loss: 0.2742 - precision: 0.9397
Epoch 5/20
125/125 [==============================] - 16s 132ms/step - loss: 0.1737 - precision: 0.9619
Epoch 6/20
125/125 [==============================] - 17s 135ms/step - loss: 0.1336 - precision: 0.9712
Epoch 7/20
125/125 [==============================] - 17s 133ms/step - loss: 0.0940 - precision: 0.9790
Epoch 8/20
125/125 [==============================] - 17s 135ms/step - loss: 0.0807 - precision: 0.9837
Epoch 9/20
125/125 [==============================] - 17s 134ms/step - loss: 0.0561 - precision: 0.9876
Epoch 10/20
125/125 [==============================] - 17s 138ms

In [49]:
score = model.evaluate(x_test, y_test)

34/34 [==============================] - 2s 23ms/step - loss: 0.9670 - precision: 0.8574


In [54]:
print("Test loss =", score[0])
print("Test precision =", score[1])

Test loss = 0.9670168161392212
Test precision = 0.8574126362800598


In [71]:
y_pred = model.predict(x_test)
print(y_pred.shape)
print(y_pred[0])
print(y_test[0])

(1065, 8)
[2.21239054e-04 5.44372275e-02 1.80053066e-05 1.93418339e-02
 1.58765152e-01 7.66991854e-01 1.16680785e-04 1.08055334e-04]
[0. 1. 0. 0. 0. 0. 0. 0.]


In [74]:
y_test_true = list(map(lambda x: np.argmax(x), y_test))
y_test_pred = list(map(lambda x: np.argmax(x), y_pred))
print(len(y_test_true))
print(len(y_test_pred))

1065
1065


In [78]:
print(y_test_true[:10])
print(y_test_pred[:10])

[1, 1, 1, 4, 3, 1, 0, 1, 1, 1]
[5, 1, 1, 4, 3, 1, 0, 1, 1, 1]


In [77]:
fbeta = fbeta_score(y_test_true, y_test_pred, beta=0.5, labels=list(range(8)), average='weighted')
print(fbeta)

0.8608755749051461
